In [1]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/nba_ai_awards/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [2]:
week = 22 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

20 March 2023 ; 23:16:26


In [3]:
features = ['VORP','PTS','WS','OWS','FG','MP','OBPM','PER','BPM','Seed']
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 10
# Propotion of maximum games played
gp = 1/3

In [4]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/smoy/smoy_data.csv')

# Load to-be-predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv') # Load up-to-date data
data = data.groupby('smoy').get_group(1)

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
0,A.J. Green,SG,23,MIL,32,0,9.4,1.5,3.4,0.440,...,3.9,115.64,111.84,3.8,3.87,115.67,111.89,3.78,1,1
1,Bobby Portis,PF,27,MIL,60,19,26.0,5.7,11.3,0.501,...,3.9,115.64,111.84,3.8,3.87,115.67,111.89,3.78,0,1
5,Jae Crowder,SF,32,MIL,11,0,18.7,2.0,4.2,0.478,...,3.9,115.64,111.84,3.8,3.87,115.67,111.89,3.78,0,1
6,Jevon Carter,PG,27,MIL,71,35,22.3,2.9,6.8,0.428,...,3.9,115.64,111.84,3.8,3.87,115.67,111.89,3.78,0,1
7,Joe Ingles,SF,35,MIL,37,0,23.1,2.4,5.6,0.435,...,3.9,115.64,111.84,3.8,3.87,115.67,111.89,3.78,0,1


In [5]:
# Set training data
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] > season_progress * gp] # At least 1/3rd of max games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [6]:
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1))
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player', 'MP', 'G', 'GS', 'Tm', 'PER', 'VORP', 'WS', 'PTS', 'Seed', 'OBPM', 'OWS', 'Share']]
res.head(5)

,Player,MP,G,GS,Tm,PER,VORP,WS,PTS,Seed,OBPM,OWS,Share
0,Malcolm Brogdon,25.9,60,0,BOS,18.1,2.460000,6.833333,14.7,3,2.4,4.373333,0.308981
1,Bobby Portis,26.0,60,19,MIL,17.9,1.503333,6.286667,13.8,1,1.1,2.870000,0.115816
2,Khris Middleton,24.0,27,13,MIL,17.2,1.214815,4.859259,14.8,1,0.8,2.429630,0.104250
3,Malik Monk,22.6,67,0,SAC,16.7,1.101493,3.549254,13.7,2,1.8,2.325373,0.098188
4,Immanuel Quickley,28.4,71,15,NYK,15.2,1.963380,6.236620,13.6,5,0.8,3.926761,0.066015


In [7]:
res.to_csv(path + f'Results/smoy/results_week_{week}.csv', index = None)